In [1]:
import os
import shutil


In [2]:
import torch
from best_library import LoadData, Preprocessing, FeatureBuilder, DatasetSplitter, ModelResnet18, HyperparameterTuner, Evaluator

In [3]:
# --- Configuration ---
DATASET_DIR = "../dataset"   # raw dataset
WORK_DIR = "../data"         # working directory for split dataset
BATCH_SIZE = 16
LR = 1e-4
EPOCHS = 5
IMG_SIZE = 224
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SAVE_PATH = "../models/best_model.pth"

if os.path.exists(WORK_DIR):
    shutil.rmtree(WORK_DIR)

os.makedirs(f"{WORK_DIR}/train/alpaca", exist_ok=True)
os.makedirs(f"{WORK_DIR}/train/not_alpaca", exist_ok=True)
os.makedirs(f"{WORK_DIR}/val/alpaca", exist_ok=True)
os.makedirs(f"{WORK_DIR}/val/not_alpaca", exist_ok=True)

os.makedirs("../models", exist_ok=True)


In [4]:
# ====================================================
# a) Preprocessing
# ====================================================

# Creating the transform that we are going to apply later to the images

preprocessor = Preprocessing(img_size=IMG_SIZE)
transform = preprocessor.get_transform()

In [5]:
# ====================================================
# b) Split dataset
# ====================================================

# Filling the dataset with 2 folders train and val, each containing alpaca and not_alpaca folders

splitter = DatasetSplitter(DATASET_DIR, WORK_DIR, train_ratio=0.8)
splitter.split()

Splitting dataset...
Dataset split complete!
Training folders: ['alpaca', 'not_alpaca']
Validation folders: ['alpaca', 'not_alpaca']


In [6]:
# ====================================================
# c) Load datasets with DataLoaders
# ====================================================

# Applying transformations to the dataset and creating two iterables for all the pictures in the training and validation

loader = LoadData(WORK_DIR, transform=transform)
train_loader, val_loader, class_names = loader.load_and_split(batch_size=BATCH_SIZE)

Classes detected: ['alpaca', 'not_alpaca']


In [7]:
# ====================================================
# d) Feature building
# ====================================================
feature_builder = FeatureBuilder()
# Example usage:
for images, labels in train_loader:
    feats = feature_builder.extract_features(images[0])

In [8]:
# ====================================================
# e) Build and train the first model
# ====================================================
model_api = ModelResnet18(device=DEVICE, class_names=class_names)
model = model_api.build_model()

model_api.train_model(model, train_loader, val_loader, epochs=EPOCHS, lr=LR)

Epoch 1/5 | Train Acc: 0.858 | Val Acc: 0.879
Epoch 2/5 | Train Acc: 0.992 | Val Acc: 0.909
Epoch 3/5 | Train Acc: 1.000 | Val Acc: 0.864
Epoch 4/5 | Train Acc: 0.996 | Val Acc: 0.894
Epoch 5/5 | Train Acc: 0.981 | Val Acc: 0.909


0.9090909090909091

In [9]:
# ====================================================
# f) Hyperparameter tuning
# ====================================================
param_grid = {
    "lr": [1e-3, 1e-4],
    "epochs": [3, 5]
}

tuner = HyperparameterTuner(param_grid, DEVICE)
best_params, best_acc = tuner.tune(train_loader, val_loader, save_path=SAVE_PATH)
print(f"Best hyperparameters: {best_params}, Best validation accuracy: {best_acc:.3f}")

Starting hyperparameter tuning with 4 combinations...

--- Trial 1/4: {'lr': 0.001, 'epochs': 3} ---
Epoch 1/3 | Train Acc: 0.759 | Val Acc: 0.636
Epoch 2/3 | Train Acc: 0.843 | Val Acc: 0.515
Epoch 3/3 | Train Acc: 0.858 | Val Acc: 0.818
Trial finished. Validation accuracy: 0.818

--- Trial 2/4: {'lr': 0.001, 'epochs': 5} ---
Epoch 1/5 | Train Acc: 0.770 | Val Acc: 0.682
Epoch 2/5 | Train Acc: 0.751 | Val Acc: 0.530
Epoch 3/5 | Train Acc: 0.916 | Val Acc: 0.758
Epoch 4/5 | Train Acc: 0.920 | Val Acc: 0.848
Epoch 5/5 | Train Acc: 0.908 | Val Acc: 0.788
Trial finished. Validation accuracy: 0.788

--- Trial 3/4: {'lr': 0.0001, 'epochs': 3} ---
Epoch 1/3 | Train Acc: 0.793 | Val Acc: 0.924
Epoch 2/3 | Train Acc: 0.981 | Val Acc: 0.909
Epoch 3/3 | Train Acc: 0.992 | Val Acc: 0.924
Trial finished. Validation accuracy: 0.924

--- Trial 4/4: {'lr': 0.0001, 'epochs': 5} ---


KeyboardInterrupt: 

In [10]:
# ====================================================
# g) Evaluate model
# ====================================================
evaluator = Evaluator(DEVICE)
best_model = model_api.load_trained_model(SAVE_PATH)
val_accuracy = evaluator.evaluate(best_model, val_loader)
print(f"Validation accuracy of the best model: {val_accuracy:.3f}")

Validation accuracy of the best model: 0.864


In [11]:
# ====================================================
# h) Predict new images 
# ====================================================
image_path = "../dog2.jpg"
label, confidence = model_api.predict(image_path, best_model, transform)
print(f"Predicted label: {label}, Confidence: {confidence:.3f}")

Predicted label: not_alpaca, Confidence: 0.966
